# Automação Web e Busca de Informações com Python

#### Desafio:

Alterar o valor final do produto de acordo com a cotação das moedas

In [28]:
# BIBLIOTECAS

from selenium import webdriver #Biblioteca de automatização web
from selenium.webdriver.common.keys import Keys # Biblioteca do selenium com as teclas do teclado
from selenium.webdriver.chrome.options import Options
import pandas as pd

In [29]:
# Passo 1: Abrir o navegador
navegador_options = Options() # Options para mandar rodar em segundo plano
navegador_options.add_argument('--headless')
navegador = webdriver.Chrome(options=navegador_options)

In [30]:
# Passo 1.2: Pesquisar e pegar a cotação do Dolar

navegador.get('https://www.google.com/')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação Dolar')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# .click() - Metodo para clicar
# .send_keys() - Metodo para escrever

# Pegando valor do google
cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)


5.4346


In [31]:
# Passo 1.3: Pesquisar e pegar a cotação do Euro

navegador.get('https://www.google.com/')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação Euro')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# .click() - Metodo para clicar
# .send_keys() - Metodo para escrever

# Pegando valor do google
cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

6.5363


In [32]:
# Passo 1.4: No melhor cambio, pegar a cotação do ouro

navegador.get('https://www.melhorcambio.com/ouro-hoje')

# .click() - Metodo para clicar
# .send_keys() - Metodo para escrever

# Pegando valor do google
cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',', '.') #Tratando a variavel para padronizar o resultado
print(cotacao_ouro)
navegador.quit() # Fechar o Navegador


308.59


In [33]:
# Passo 2: Importar os dados da tabela
tabela_produtos = pd.read_excel('Produtos.xlsx')
display(tabela_produtos)
tabela_produtos.info()

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Produtos             7 non-null      object 
 1   Preço Base Original  7 non-null      float64
 2   Moeda                7 non-null      object 
 3   Cotação              7 non-null      int64  
 4   Preço Base Reais     7 non-null      float64
 5   Ajuste               7 non-null      float64
 6   Preço Final          7 non-null      float64
dtypes: float64(4), int64(1), object(2)
memory usage: 520.0+ bytes


In [34]:
# Passo 3: Atualizar a cotação na tabela
tabela_produtos.loc[ tabela_produtos['Moeda']== 'Dólar', 'Cotação'] = float(cotacao_dolar)
tabela_produtos.loc[ tabela_produtos['Moeda']== 'Euro', 'Cotação'] = float(cotacao_euro)
tabela_produtos.loc[ tabela_produtos['Moeda']== 'Ouro', 'Cotação'] = float(cotacao_ouro)
# metodo loc de localizar e recebe dois parametros, linha e coluna

tabela_produtos['Preço Base Reais'] = tabela_produtos['Preço Base Original'] * tabela_produtos['Cotação']
tabela_produtos['Preço Final'] = tabela_produtos['Preço Base Reais'] * tabela_produtos['Ajuste']
tabela_produtos['Preço Final'] = tabela_produtos['Preço Final'].map("{:.2f}".format)

display(tabela_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5.4346,5434.545654,1.40,7608.36
1,Carro Renault,4500.00,Euro,6.5363,29413.350000,2.00,58826.70
2,Notebook Dell,899.99,Dólar,5.4346,4891.085654,1.70,8314.85
3,IPhone,799.00,Dólar,5.4346,4342.245400,1.70,7381.82
4,Carro Fiat,3000.00,Euro,6.5363,19608.900000,1.90,37256.91
5,Celular Xiaomi,480.48,Dólar,5.4346,2611.216608,2.00,5222.43
6,Joia 20g,20.00,Ouro,308.5900,6171.800000,1.15,7097.57


In [35]:
# Passo 4: Exportar a base de dados
tabela_produtos.to_excel('Produtos Atualizados.xlsx', index=False)
#index false é para retirar o indice que o pandas cria